In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import Libraries

In [ ]:
pip install pytesseract

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import pytesseract as pt
import plotly.express as px
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet #Efficient way to work with XML files

from glob import glob
from skimage import io
from shutil import copy
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Lebelling The Images

## PARSING DATA FROM XML AND CONVERTING IT INTO CSV

### Parsing Data from XML

In [ ]:
path = glob('/content/drive/MyDrive/Colab Notebooks/11 - Kaggle Practicing/7 - Automatic Number Plate Recognition/images/*.xml')
labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[]) #The labels_dict is an Unstructured Data in this step
for filename in path:
  info = xet.parse(filename)
  root = info.getroot()
  member_object = root.find('object')
  labels_info = member_object.find('bndbox')

  # Ya
  # labels_info = xet.parse(filename).getroot().find('object').find('bndbox')

  xmin = int(labels_info.find('xmin').text)
  xmax = int(labels_info.find('xmax').text)
  ymin = int(labels_info.find('ymin').text)
  ymax = int(labels_info.find('ymax').text)

  labels_dict['filepath'].append(filename)
  labels_dict['xmin'].append(xmin)
  labels_dict['xmax'].append(xmax)
  labels_dict['ymin'].append(ymin)
  labels_dict['ymax'].append(ymax)

In [ ]:
df = pd.DataFrame(labels_dict)
#The labels_dict is an structured Data in this step
df.to_csv('labels.csv',index=False)
df.head()

,filepath,xmin,xmax,ymin,ymax
0,/content/drive/MyDrive/Colab Notebooks/11 - Ka...,192,364,279,319
1,/content/drive/MyDrive/Colab Notebooks/11 - Ka...,175,290,228,255
2,/content/drive/MyDrive/Colab Notebooks/11 - Ka...,144,277,216,252
3,/content/drive/MyDrive/Colab Notebooks/11 - Ka...,150,348,255,296
4,/content/drive/MyDrive/Colab Notebooks/11 - Ka...,383,491,212,248


With the above code, we successfully extract the diagonal position of each image and convert the data from an unstructured to a structured format.You can have A look data above. Now also extract the respective image filename of the XML.

### Extract the Respective image filename of XML Files

In [ ]:
filename = df['filepath'][0]
def getFilename(filename):
  filename_image = xet.parse(filename).getroot().find('filename').text
  filepath_image = os.path.join('/content/drive/MyDrive/Colab Notebooks/11 - Kaggle Practicing/7 - Automatic Number Plate Recognition/images',filename_image)
  return filepath_image
getFilename(filename)

'/content/drive/MyDrive/Colab Notebooks/11 - Kaggle Practicing/7 - Automatic Number Plate Recognition/images/N245.jpeg'

In [ ]:
image_path = list(df['filepath'].apply(getFilename))
image_path[:10] #Random check

['/content/drive/MyDrive/Colab Notebooks/11 - Kaggle Practicing/7 - Automatic Number Plate Recognition/images/N245.jpeg',
 '/content/drive/MyDrive/Colab Notebooks/11 - Kaggle Practicing/7 - Automatic Number Plate Recognition/images/N97.jpeg',
 '/content/drive/MyDrive/Colab Notebooks/11 - Kaggle Practicing/7 - Automatic Number Plate Recognition/images/N151.jpeg',
 '/content/drive/MyDrive/Colab Notebooks/11 - Kaggle Practicing/7 - Automatic Number Plate Recognition/images/N141.jpeg',
 '/content/drive/MyDrive/Colab Notebooks/11 - Kaggle Practicing/7 - Automatic Number Plate Recognition/images/N51.jpeg',
 '/content/drive/MyDrive/Colab Notebooks/11 - Kaggle Practicing/7 - Automatic Number Plate Recognition/images/N123.jpeg',
 '/content/drive/MyDrive/Colab Notebooks/11 - Kaggle Practicing/7 - Automatic Number Plate Recognition/images/N36.jpeg',
 '/content/drive/MyDrive/Colab Notebooks/11 - Kaggle Practicing/7 - Automatic Number Plate Recognition/images/N241.jpeg',
 '/content/drive/MyDrive/Co